# Basic Test

In [1]:
from Go2Py.sim.mujoco import Go2Sim
import numpy as np

In [2]:
robot = Go2Sim()
robot.standUpReset()

In [3]:
from Go2Py.control.walk_these_ways import *
# checkpoint_path = "/home/rstaion/projects/rooholla/locomotion/Go2Py/Go2Py/assets/checkpoints/walk_these_ways/"
checkpoint_path = "/home/rstaion/projects/rooholla/walk-these-ways/runs/gait-conditioned-agility/pretrain-v0/train/025417.456545"


cfg = loadParameters(checkpoint_path)
policy = Policy(checkpoint_path)
command_profile = CommandInterface()

In [4]:
agent = WalkTheseWaysAgent(cfg, command_profile, robot)
agent = HistoryWrapper(agent)

p_gains: [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]


In [5]:
control_dt = cfg["control"]["decimation"] * cfg["sim"]["dt"]
simulation_dt = robot.dt
obs = agent.reset()

In [6]:
robot.reset()
obs = agent.reset()
for i in range(50000):
    policy_info = {}
    action = policy(obs, policy_info)
    if i % (control_dt // simulation_dt) == 0:
        obs, ret, done, info = agent.step(action)
    robot.step()
    command_profile.yaw_vel_cmd = 0.0
    command_profile.x_vel_cmd = 0.0
    command_profile.y_vel_cmd = 0.0
    time.sleep(robot.dt/4)

frq: 15.967047985229458 Hz
frq: 42.47353444522081 Hz
frq: 43.69977078558033 Hz
frq: 40.20806211954177 Hz


# Full System With FSM

In [1]:
from Go2Py.robot.fsm import FSM
from Go2Py.robot.remote import KeyboardRemote
from Go2Py.robot.safety import SafetyHypervisor
from Go2Py.sim.mujoco import Go2Sim
from Go2Py.control.walk_these_ways import *
import numpy as np

In [2]:
robot = Go2Sim()

In [3]:
class walkTheseWaysController:
    def __init__(self, robot, remote, checkpoint):
        self.remote = remote
        self.robot = robot
        self.cfg = loadParameters(checkpoint)
        self.policy = Policy(checkpoint)
        self.command_profile = CommandInterface()
        self.agent = WalkTheseWaysAgent(self.cfg, self.command_profile, self.robot)
        self.agent = HistoryWrapper(self.agent)
        self.init()

    def init(self):
        self.obs = self.agent.reset()
        self.policy_info = {}
        self.command_profile.yaw_vel_cmd = 0.0
        self.command_profile.x_vel_cmd = 0.0
        self.command_profile.y_vel_cmd = 0.0
        self.command_profile.stance_width_cmd=0.25
        self.command_profile.footswing_height_cmd=0.08
        self.command_profile.step_frequency_cmd = 3.0
        self.command_profile.body_height_cmd = 0.5

    def update(self, robot, remote):
        action = self.policy(self.obs, self.policy_info)
        self.obs, self.ret, self.done, self.info = self.agent.step(action)


In [4]:
remote = KeyboardRemote()
robot.standUpReset()
safety_hypervisor = SafetyHypervisor(robot)

In [5]:
checkpoint = "/home/rstaion/projects/rooholla/locomotion/Go2Py/Go2Py/assets/checkpoints/walk_these_ways/"
# checkpoint = "/home/rstaion/projects/rooholla/walk-these-ways/runs/gait-conditioned-agility/pretrain-v0/train/025417.456545"
controller = walkTheseWaysController(robot, remote, checkpoint)
fsm = FSM(robot, remote, safety_hypervisor, user_controller_callback=controller.update)

p_gains: [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]


frq: 0.10237921921805228 Hz
frq: 46.14348108298404 Hz
frq: 44.397794032030994 Hz
frq: 45.603148715941465 Hz
frq: 45.86595514341641 Hz
frq: 44.16591026356524 Hz
frq: 45.82636627843454 Hz
frq: 45.51950772169346 Hz


In [7]:
controller.command_profile.pitch_cmd=0.0
controller.command_profile.body_height_cmd=0.0
controller.command_profile.footswing_height_cmd=0.08
controller.command_profile.roll_cmd=0.0
controller.command_profile.stance_width_cmd=0.2
controller.command_profile.x_vel_cmd=-0.2
controller.command_profile.y_vel_cmd=0.01
controller.command_profile.setGaitType("trotting")

frq: 45.741406386320016 Hz
frq: 44.6901430961184 Hz
frq: 44.451434445775085 Hz
frq: 45.19042385846963 Hz
frq: 45.1087737411542 Hz
frq: 46.690013692073066 Hz
frq: 45.09228519824546 Hz
frq: 45.70053825535531 Hz
frq: 46.01388873651992 Hz
frq: 44.94924554183813 Hz
frq: 45.49383372200228 Hz
frq: 44.522684330085134 Hz
frq: 44.42130458266699 Hz
frq: 43.71525649844704 Hz
frq: 44.37383889464886 Hz


In [ ]:
controller.command_profile.pitch_cmd=0.0
controller.command_profile.body_height_cmd=0.0
controller.command_profile.footswing_height_cmd=0.08
controller.command_profile.roll_cmd=0.0
controller.command_profile.stance_width_cmd=0.2
controller.command_profile.x_vel_cmd=-0.2
controller.command_profile.y_vel_cmd=0.01
controller.command_profile.setGaitType("trotting")

In [ ]:
fsm.close()